## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [538]:
path = '../03-corpus/2-data/1-fr/'
acteur = "asso_ordres"

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f, sep=';')[["url", "text"]]

csv

,url,text
0,http://amoe.qc.ca/\n,Association des médecins omnipraticiens de l'E...
1,http://amol.ca/\n,AMOL - PORTAIL MÉDECINS MEMBRES Suivez-nous! M...
2,http://amooq.ca/\n,AMOOQ – Association des Médecins Omnipraticien...
3,http://amoq.ca/\n,:: AMOQ | Association des Médecins Omnipratici...
4,http://amoq.ca/amoq.html\n,:: AMOQ | Association des Médecins Omnipratici...
...,...,...
33882,https://www1.otstcfq.org/wp-json/wp/v2/taxocsv...,"{""code"":""itsec_rest_api_access_restricted"",""me..."
33883,https://www1.otstcfq.org/wp-json/wp/v2/taxocsv...,"{""code"":""itsec_rest_api_access_restricted"",""me..."
33884,https://www1.otstcfq.org/wp-json/wp/v2/taxocsv...,"{""code"":""itsec_rest_api_access_restricted"",""me..."
33885,https://www1.otstcfq.org/wp-json/wp/v2/taxocsv...,"{""code"":""itsec_rest_api_access_restricted"",""me..."


In [539]:
liste = [lien.replace('//', "/") for lien in csv['url'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien and not 'nouvelles' in lien]

liste

['http:/amoe.qc.ca/\n',
 'http:/amol.ca/\n',
 'http:/amooq.ca/\n',
 'http:/amoq.ca/\n',
 'http:/amoq.ca/amoq.html\n',
 'http:/amoq.ca/bottin.html\n',
 'http:/amoq.ca/contact.html\n',
 'http:/amoq.ca/formation.html\n',
 'http:/amoq.ca/formulaires.html\n',
 'http:/amoq.ca/index.html\n',
 'http:/amoq.ca/liens.html\n',
 'http:/amoq.ca/ressources.html\n',
 'http:/amoq.ca/services_fede.html\n',
 'http:/amoq.ca/services_membres.html\n',
 'http:/amoq.ca/ShowClinique.php\n',
 'http:/amoq.ca/ShowTdns.php\n',
 'http:/amoq.ca/tdns.html\n',
 'http:/amosl.ca/\n',
 'http:/opdq.org/\n',
 'http:/www.acmdp.qc.ca/\n',
 'http:/www.aiispq.org/\n',
 'http:/www.aiiuq.qc.ca/\n',
 'http:/www.aipi.qc.ca/\n',
 'http:/www.aipsq.com/\n',
 'http:/www.allerg.qc.ca/\n',
 'http:/www.allerg.qc.ca/index.php\n',
 'http:/www.allerg.qc.ca/index.php?prof=TRUE\n',
 'http:/www.allerg.qc.ca/Information_allergique/1_1_specialite.html\n',
 'http:/www.allerg.qc.ca/Information_allergique/1_1_specialite_en.html\n',
 'http:/www.alle

In [540]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""
liens_1N = [lien for lien in liste if len(lien.split('/')) > 2]
liens_2N = [lien for lien in liste if len(lien.split('/')) > 3]
liens_3N = [lien for lien in liste if len(lien.split('/')) > 4]
liens_4N = [lien for lien in liste if len(lien.split('/')) > 5]
liens_5N = [lien for lien in liste if len(lien.split('/')) > 6]
liens_6N = [lien for lien in liste if len(lien.split('/')) > 7]
liens_7N = [lien for lien in liste if len(lien.split('/')) > 8]

In [562]:
regroupements = {}

for lien in liens_7N:
    tag = lien.split('/')[8]
    if tag in regroupements:
        regroupements[tag].append(lien)
        
    else:
        if len(tag)>1:
            regroupements[tag] = []

tags = list(regroupements.keys())

In [563]:
# On garde seulement les sections qui contiennent au moins 5 sous pages
tags_retenus = [tag for tag in tags if len(regroupements[tag]) >=5]

In [564]:
print('Ça nous donne {} tags.'.format(len(tags_retenus)))
tags_retenus

Ça nous donne 55 tags.


['fr-CA',
 'Default.aspx\n',
 'language',
 'en-US',
 '%22\n',
 'archive',
 'leadership.html\n',
 'aide-financiere.html\n',
 'cyberjournl.html\n',
 'finances.html\n',
 'jeunesse.html\n',
 'concours-innovation-clinique.html\n',
 'formation.html\n',
 'mot-de-la-presidente.html\n',
 'excellence-des-soins.html\n',
 'rayonnement-international.html\n',
 'recipiendaires-2003',
 'recipiendaires-2004',
 'recipiendaires-2005',
 'recipiendaires-2006',
 'recipiendaires-2007',
 'recipiendaires-2008',
 'recipiendaires-2009',
 'recipiendaires-2010',
 'mot_presidente.html\n',
 'courriel.html\n',
 'faq.html\n',
 'conferences',
 'ateliers-thematiques',
 'prix',
 'prix-et-bourses',
 'comites.html\n',
 'comites',
 'contact.html\n',
 'prix-et-bourses.html\n',
 'publications.html\n',
 'reglement.html\n',
 'assemblee-generale.html\n',
 'conseil-de-section.html\n',
 'comite',
 'tous-les-recipiendaires',
 'recipiendaires',
 'recipiendaires-2011',
 'recipiendaires-2013',
 'recipiendaires-2014',
 'recipiendaires-

In [565]:
import os, shutil
from pathlib import Path

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/'

#Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags_retenus:
    file_name =  acteur + "_" + tag.strip('\n')

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    try:
        data = csv[csv["url"].str.match('.*[\-\_\/]'+tag+'.*')]
        data.to_csv(folder_path + file_name + ".csv") #encoding='ISO-8859-1'
    except Exception as e:
        print(tag, str(e))
                    

In [566]:
with open(folder_path + 'sous-corpus.txt', 'w') as f:
    for file in os.listdir(folder_path):
        f.write(file + '\n')

Après avoir manuellement indexé

In [582]:
from pandas import read_csv
filepath = '../03-corpus/2-sous-corpus/0-semi-automatic_tags.csv'
with open(filepath, 'r', encoding = 'utf-8') as f:
    csv = read_csv(f, sep=';')
    
csv = csv[csv['Sous-corpus'] != 'x']
csv = csv[csv['Sous-corpus'] != 'X']

csv

,Regroupement dans l'arborescence du site,Sous-corpus,D1,Remarque
0,chum_axes-de-recherche.csv,Axe_recherche,NaN,NaN
1,chum_cancer-du-sein.csv,Cancer,NaN,NaN
2,chum_cancer-oculaire.csv,Cancer,NaN,NaN
3,chum_carrieres-crchum.csv,Carrières,NaN,NaN
5,chum_centre-cardiovasculaire.csv,Cardiologie,NaN,NaN
...,...,...,...,...
2073,asso_ordres_utilisation-de-lentretien-motivati...,Kinésiologie,NaN,NaN
2084,asso_ordres_www-pp.infoiiq.org.csv,Nursing,NaN,NaN
2086,asso_ordres_www.infoiiq.org.csv,Nursing,NaN,NaN
2087,asso_ordres_www.oiiq.org.csv,Nursing,NaN,NaN


In [583]:
tags = list(set([tag for tag in csv['Sous-corpus'].tolist()]))

tags

['Jeunesse',
 'Enseignement',
 'Prothèses',
 'Kinésiologie',
 'Vaccination',
 'Fondation',
 "Services préhospitaliers d'urgence",
 'DI / TSA',
 'Santé Publique',
 'Télésanté',
 'Zoonoses',
 'Promotion de la santé',
 'Traumatologie',
 'Hygiène',
 'Pharmacologie',
 'Violence / agressions',
 'covid-19',
 'Pédiatrie',
 'Soins paliatifs',
 'Système sanguin',
 'Carrières',
 'Massothérapie',
 'Soins 1e ligne',
 'Geriatrie',
 'Accès aux services',
 'Prévention_incendies',
 'Sexualité',
 'Programmes',
 'Maladies chroniques',
 "Mesures d'urgence",
 'Dependances',
 'Ophtalmologie',
 'Orthophonie',
 'Sante travail',
 'Aînés',
 "Soins préhospitaliers d'urgence",
 'Laboratoires',
 'Famille',
 'Audiologie',
 'Pratiques préventives',
 'Inhalotherapie',
 'Matériel médical',
 'Professionnels',
 'Environnement ',
 'Suicide',
 'Santé publique',
 'Psychologie',
 'Services ambulatoires',
 'Inconduites sexuelles',
 'Immunologie',
 'Cancer',
 'Diagnostique',
 'Environnement',
 'DeficienceHandicap',
 'Itinéran

In [584]:
regroupements = {tag : [] for tag in tags}
for tag in tags:
        tuples = csv[csv['Sous-corpus'] == tag]
        liste = [t.strip('\n') for t in tuples["Regroupement dans l'arborescence du site"].tolist()]
        regroupements[tag] = liste


In [585]:
regroupements

{'Jeunesse': ['cisss_ciusss_adolescents.csv',
  'cisss_ciusss_adoption.csv',
  'cisss_ciusss_centre-de-protection-et-de-readaptation-pour-les-jeunes-et-les-meres-en-difficul.csv',
  'cisss_ciusss_Centres_de_r%C3%A9adaptation_pour_jeunes_et_m%C3%A8res_en_difficult%C3%A9.csv',
  'cisss_ciusss_DPJ2020.csv',
  'cisss_ciusss_enquete-quebecoise-sur-la-sante-des-jeunes-du-secondaire-eqsjs-2016-2017-pour-le.csv',
  'cisss_ciusss_Jeunes-secondaire.csv',
  'cisss_ciusss_Mere-enfant_famille_jeunesse_1ere_2eme_ligne_0-17_ans_.csv',
  'cisss_ciusss_obtenir-des-services-pour-les-jeunes-et-leur-famille.csv',
  'cisss_ciusss_outils-pour-les-intervenants-jeunesse.csv',
  'cisss_ciusss_protection-de-la-jeunesse.csv',
  'cisss_ciusss_protection-et-readaptation-pour-les-jeunes-en-difficulte-dadaptation.csv',
  'cisss_ciusss_services-en-jeunesse.csv',
  'inesss_jeunesse.csv',
  'msss_jeunesse.csv'],
 'Enseignement': ['cisss_ciusss_enseignement-et-stages.csv',
  'cisss_ciusss_enseignement-medical.csv',
  'c

In [589]:
import pandas
from slugify import slugify

# Créer de nouveaux CSV contenant les données regroupées selon le tag qui leur est associé
path='../03-corpus/2-sous-corpus/'

for tag in regroupements:

    # Aller chercher les éléments correspondant au sous-corpus dans le csv principal et les copier dans un nouveau CSV 
    dfs = []
    
    for doc in regroupements[tag]:
        with open(path + doc, 'r', encoding='utf-8') as f:
            csv = read_csv(f)
            dfs.append(csv)

    # Concatenate all data into one DataFrame
    try:
        file_path = path + 'regroupements/others/' + slugify(str(tag)) + '.csv'
        big_frame = pandas.concat(dfs, ignore_index=True)
        big_frame.to_csv(file_path)
        print(tag + "\t" + str(len(big_frame)))

    except Exception as e:
        print(tag, str(e))



Jeunesse	384
Enseignement	211
Prothèses	157
Kinésiologie	236
Vaccination	992
Fondation	79
Services préhospitaliers d'urgence	42
DI / TSA	137
Santé Publique	7280
Télésanté	120
Zoonoses	48
Promotion de la santé	1144
Traumatologie	57
Hygiène	41
Pharmacologie	665
Violence / agressions	167
covid-19	6449
Pédiatrie	29
Soins paliatifs	215
Système sanguin	18
Carrières	1690
Massothérapie	290
Soins 1e ligne	44
Geriatrie	191
Accès aux services	117
Prévention_incendies	59
Sexualité	38
Programmes	445
Maladies chroniques	157
Mesures d'urgence	16
Dependances	448
Ophtalmologie	375
Orthophonie	158
Sante travail	108
Aînés	1918
Soins préhospitaliers d'urgence	83
Laboratoires	96
Famille	236
Audiologie	206
Pratiques préventives	40
Inhalotherapie	62
Matériel médical	56
Professionnels	9329
Environnement 	34
Suicide	94
Santé publique	8
Psychologie	125
Services ambulatoires	35
Inconduites sexuelles	8
Immunologie	191
Cancer	1242
Diagnostique	10
Environnement	347
DeficienceHandicap	636
Itinérance	114
Gastroentero